In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier as KNN
import pickle
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('04_22.csv')
df[0:5]

,Unnamed: 0,pastHP-AP,pastHG-AG,pastAGC-HGC,pastHS-AS,pastHST-AST,pastASC-HSC,pastASTC-HSTC,HomeFullPoint,AwayFullPoint,HomeGoal,AwayGoal,HomeConceded,AwayConceded,HomeAvgPoint,AwayAvgPoint,FTR
0,30,1.199006,-0.325458,1.672309,0.092964,0.072379,0.371667,0.741030,3.063633,-0.719519,-0.001163,-0.030735,-3.109829,1.543916,3.950040,-0.934438,H
1,31,0.459169,0.850221,-1.134070,0.545853,0.878015,-1.007148,-2.061876,0.253267,-0.719519,1.293817,-0.674108,2.663004,-0.768541,0.295855,-0.934438,H
2,32,-2.130261,-1.501137,-0.733159,-1.175124,-2.183401,-0.087938,-0.248231,-1.932574,1.150657,-1.296144,1.256011,-0.030984,0.773097,-2.546289,1.511995,H
3,33,1.199006,1.634007,0.068664,-0.178769,-0.411002,0.831272,0.741030,-0.058996,-1.966302,1.941308,-1.317481,2.278149,1.543916,-0.110165,-2.565393,H
4,34,-1.760342,-1.893029,-0.332247,-0.903391,-2.022274,-2.477883,-2.721384,-1.308048,3.020833,-0.001163,5.116251,0.738727,0.002278,-1.734247,3.958427,A


In [3]:
features = ['pastHP-AP','pastHG-AG','pastAGC-HGC','pastHS-AS', 'pastHST-AST', 'pastASC-HSC', 'pastASTC-HSTC', 
              'HomeFullPoint','AwayFullPoint','HomeGoal','AwayGoal','HomeConceded','AwayConceded','HomeAvgPoint','AwayAvgPoint', 'FTR']
data = df[features]
data[:5]

,pastHP-AP,pastHG-AG,pastAGC-HGC,pastHS-AS,pastHST-AST,pastASC-HSC,pastASTC-HSTC,HomeFullPoint,AwayFullPoint,HomeGoal,AwayGoal,HomeConceded,AwayConceded,HomeAvgPoint,AwayAvgPoint,FTR
0,1.199006,-0.325458,1.672309,0.092964,0.072379,0.371667,0.741030,3.063633,-0.719519,-0.001163,-0.030735,-3.109829,1.543916,3.950040,-0.934438,H
1,0.459169,0.850221,-1.134070,0.545853,0.878015,-1.007148,-2.061876,0.253267,-0.719519,1.293817,-0.674108,2.663004,-0.768541,0.295855,-0.934438,H
2,-2.130261,-1.501137,-0.733159,-1.175124,-2.183401,-0.087938,-0.248231,-1.932574,1.150657,-1.296144,1.256011,-0.030984,0.773097,-2.546289,1.511995,H
3,1.199006,1.634007,0.068664,-0.178769,-0.411002,0.831272,0.741030,-0.058996,-1.966302,1.941308,-1.317481,2.278149,1.543916,-0.110165,-2.565393,H
4,-1.760342,-1.893029,-0.332247,-0.903391,-2.022274,-2.477883,-2.721384,-1.308048,3.020833,-0.001163,5.116251,0.738727,0.002278,-1.734247,3.958427,A


In [4]:
X = data.drop(['FTR'], axis=1)
Y = data[['FTR']]
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
Y[:5]

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


array([2, 2, 2, 2, 0])

In [5]:
# lay index de chia train_test
split_train = 5950

X_train, X_test = X[:split_train], X[split_train: ]
y_train, y_test = Y[:split_train], Y[split_train: ]

In [6]:
model_ann_relu = keras.models.load_model('model_ann_relu')

model_xgb_6_depth = XGBClassifier()
model_xgb_6_depth.load_model("model_max_depth_6_tree_20.xgb")

with open('model_manhattan_metric_k_700.pkl', 'rb') as f:
    model_knn_manhattan = pickle.load(f)

OSError: No file or directory found at model_ann_relu

In [ ]:
def compare():
    ann_predict = model_ann_relu.evaluate(X_test, y_test)
    # print("ann: ", ann_predict[1])
    xgb_predict = accuracy_score(y_test, model_xgb_6_depth.predict(X_test))
    # print("xgb: ", xgb_predict)
    knn_predict = accuracy_score(y_test, model_knn_manhattan.predict(X_test))
    # print("knn: ", knn_predict)

    lables = ['ANN', 'XGB', 'KNN']
    colors = ['skyblue', "magenta", 'green']
    #subplot1
    f, (ax1) = plt.subplots(1, 1, figsize=(7,3))
    ax1.set_title('Compare model')

    accuracy = [ann_predict[1], xgb_predict, knn_predict]

    ax1.set(ylabel='test accuracy')
    ax1.bar(lables, accuracy, color=colors)
    ax1.set_ylim(bottom=0, top=1)
    for x, y in enumerate(accuracy):
        ax1.text(x-0.1, y+0.002, f"{round(y, 3):.3f}", color = 'black')

In [ ]:
compare()